# THESIS NOTEBOOK - infer.py

### Libraries are imported here:

In [49]:
import numpy as np
from PIL import Image

import caffe
import vis

# own libraries
import colorgram
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import random
import shutil
import time
import datetime
import math
import sys

import score
import voc_layers

### Constant and global variables:

In [50]:
# own code - Jasper
PRINT_SECONDS = 0.2
REVIEW_SECONDS = 5
REST_SECONDS = 10
ERROR_ABOVE = "Image {height, width} has above 1000 pixels"
PASS_BELOW = "Image {height, width} has below 1000 pixels"
OUTPUT_FOLDER = "output_score" # no slashes both first and last
OUTPUT_FOLDER_VALIDATION = "output_validate"
dm = "" # global dimensions string
JPG_FILETYPE = ".jpg"
PNG_FILETYPE = ".png"
loss = 0
over_acc_scores = []
mean_acc_scores = []
iu_scores = []
freq_scores = []
session_count = 0

### Function section here:

#### trackSession

In [51]:
def trackSession():
    global session_count
    if session_count == 0:
        session_count += 1
        return True
    else:
        return False
    return False

#### setSession

In [52]:
def setSession(file, flag=True):
    now = datetime.datetime.now()
    if flag:
        file.write("\n-------------------- SESSION - {} -------------------------\n".format(now.strftime("%Y-%m-%d %H:%M")))
    else:
        file.write("-------------------- SESSION - {} -------------------------\n".format(now.strftime("%Y-%m-%d %H:%M")))

#### endSession

In [53]:
def endSession(flag=True):
    now = datetime.datetime.now()
    if flag:
        return "\n-------------------- END SESSION - {} -------------------------\n".format(now.strftime("%Y-%m-%d %H:%M"))
    else:
        return "-------------------- END SESSION - {} -------------------------\n".format(now.strftime("%Y-%m-%d %H:%M"))
    return ""

#### delayPrint

In [54]:
def delayPrint(string, seconds): # n seconds delay printing
    time.sleep(seconds)
    exportLogs(string)
    print(string)

#### getPaintings

In [55]:
def getPaintings(path):
    paintings = [f.split(JPG_FILETYPE)[0] for f in listdir(path) if isfile(join(path, f))] # splits only for .jpg images
    paintings.sort()
    return paintings

#### getLabels

In [56]:
def getLabels(path):
    paintings = [f.split(PNG_FILETYPE)[0] for f in listdir(path) if isfile(join(path, f))] # splits only for .png images
    paintings.sort()
    return paintings

#### exportLogs

In [57]:
def exportLogs(logs, f="demo/logs.log"):
    logs += "\n"
    if(isfile(f)):
        file = open(f, "a")
        if trackSession():
            setSession(file)
        file.write(logs)
        file.close()
    else:
        print("Log file does not exist!")
        print("Creating {} file...".format(f))
        file = open(f, "a+")
        if trackSession():
            setSession(file)
        file.write(logs)
        file.close()

#### createCurrentLog

In [71]:
def createCurrentLog(fp, output_folder=OUTPUT_FOLDER):
    filepath = fp.split("/")
    f = "demo/{}/{}.log".format(output_folder, filepath[len(filepath) - 1].split(JPG_FILETYPE)[0])
    if(isfile(f)):
        delayPrint("Resuming {} file".format(f), PRINT_SECONDS)
        file = open(f, "w")
        setSession(file, False)
        file.close()
    else:
        print("{} log file does not exist!".format(f))
        print("Creating {} file...".format(f))
        file = open(f, "w+")
        setSession(file, False)
        file.close()

#### readResume

In [59]:
def readResume(f="demo/resume.txt"):
    fp_resume = ""
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    if(isfile(f)):
        file = open(f, "r")
        delayPrint("Reading file...", PRINT_SECONDS)
        fp_resume = file.read()
        delayPrint("Last segmented image: {}".format(fp_resume.rstrip()), PRINT_SECONDS)
        delayPrint("Closing file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("File does not exist!", PRINT_SECONDS)
    return fp_resume

#### writeResume

In [60]:
def writeResume(current_painting_path, f="demo/resume.txt"):
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    if(isfile(f)):
        file = open(f, "w+")
        delayPrint("Saving last segmented image path...", PRINT_SECONDS)
        delayPrint("Writing file...", PRINT_SECONDS)
        file.write(current_painting_path)
        delayPrint("Closing file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("File does not exist!", PRINT_SECONDS)

#### writeErrorFile

In [61]:
def writeErrorFile(current_painting_path, error="", f="demo/error.log"):
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    if(isfile(f)):
        file = open(f, "a")
        delayPrint("Writing error file...", PRINT_SECONDS)
        file.write("{} - {}\n".format(current_painting_path, error))
        delayPrint("Closing error file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("{} log file does not exist!".format(f), PRINT_SECONDS)
        delayPrint("Creating {} file...".format(f), PRINT_SECONDS)
        delayPrint("Writing error file...", PRINT_SECONDS)
        file = open(f, "a+")
        file.write("{} - {}\n".format(current_painting_path, error))
        delayPrint("Closing error file...", PRINT_SECONDS)
        file.close()

#### reshapeInputLayer

In [62]:
def reshapeInputLayer(img, LINE_NUMBER=7, f="voc-fcn8s/test.prototxt"):
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    # LINE_NUMBER = 7
    width, height = img.size
    if(isfile(f)):
        with open(f, "r") as file:
            delayPrint("Reading file...", PRINT_SECONDS)
            data = file.readlines()
            delayPrint("Reshaping input layer...", PRINT_SECONDS)
            data[LINE_NUMBER] = "    shape { dim: 1 dim: 3 dim: %s dim: %s }\n"%(height, width)
            delayPrint(data[LINE_NUMBER], PRINT_SECONDS)
        with open(f, "w+") as file:
            delayPrint("Writing file...", PRINT_SECONDS)
            file.writelines(data)
        delayPrint("Closing file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("File does not exist!", PRINT_SECONDS)

#### reshapeLabelLayer

In [63]:
def reshapeLabelLayer(img, LINE_NUMBER=7, f="voc-fcn8s/test.prototxt"):
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    # LINE_NUMBER = 7
    width, height = img.size
    if(isfile(f)):
        with open(f, "r") as file:
            delayPrint("Reading file...", PRINT_SECONDS)
            data = file.readlines()
            delayPrint("Reshaping label layer...", PRINT_SECONDS)
            data[LINE_NUMBER] = "    shape { dim: 1 dim: %s dim: %s }\n"%(height, width)
            delayPrint(data[LINE_NUMBER], PRINT_SECONDS)
        with open(f, "w+") as file:
            delayPrint("Writing file...", PRINT_SECONDS)
            file.writelines(data)
        delayPrint("Closing file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("File does not exist!", PRINT_SECONDS)

#### checkImageSize1000000

In [64]:
def checkImageSize1000000(img):
    delayPrint("Checking if image {width, height} has 1,000,000 and above pixels...", PRINT_SECONDS)
    width, height = img.size
    dimensions = " - ({} x {})".format(height, width)
    global dm
    dm = dimensions
    ONE_MIL = 1000000
    delayPrint("Images (H x W) has dimensions {} x {}".format(height, width), PRINT_SECONDS)
    if((width * height) >= ONE_MIL):
        delayPrint(ERROR_ABOVE+dimensions, PRINT_SECONDS)
        return True
    else:
        delayPrint(PASS_BELOW+dimensions, PRINT_SECONDS)
        return False
    return False

#### validationResultsLog

In [65]:
def validationResultsLog(current_painting_path, results, f="demo/results.log"):
    global loss
    global over_acc_scores
    global mean_acc_scores
    global iu_scores
    global freq_scores
    loss += results['loss']
    over_acc_scores.append(results['over_acc'])
    mean_acc_scores.append(results['mean_acc'])
    iu_scores.append(results['iu'])
    freq_scores.append(results['freq'])
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    if(isfile(f)):
        file = open(f, "a")
        delayPrint("Writing results file...", PRINT_SECONDS)
        file.write("{}\n".format(current_painting_path))
        file.write("\t\tLoss: {}\n".format(results['loss']))
        file.write("\t\tOverall accuracy: {}\n".format(results['over_acc']))
        file.write("\t\tMean accuracy: {}\n".format(results['mean_acc']))
        file.write("\t\tMean IU: {}\n".format(results['iu']))
        file.write("\t\tFwavacc accuracy: {}\n".format(results['freq']))
        file.write("\n")
        delayPrint("Closing results file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("{} log file does not exist!".format(f), PRINT_SECONDS)
        delayPrint("Creating {} file...".format(f), PRINT_SECONDS)
        delayPrint("Writing results file...", PRINT_SECONDS)
        file = open(f, "a+")
        file.write("{}\n".format(current_painting_path))
        file.write("\t\tLoss: {}\n".format(results['loss']))
        file.write("\t\tOverall accuracy: {}\n".format(results['over_acc']))
        file.write("\t\tMean accuracy: {}\n".format(results['mean_acc']))
        file.write("\t\tMean IU: {}\n".format(results['iu']))
        file.write("\t\tFwavacc accuracy: {}\n".format(results['freq']))
        file.write("\n")
        delayPrint("Closing results file...", PRINT_SECONDS)
        file.close()

#### computeMeanScore

In [66]:
def computeMeanScore(f="demo/results.log"):
    global loss
    global over_acc_scores
    global mean_acc_scores
    global iu_scores
    global freq_scores
    over_acc_scores = np.mean(over_acc_scores)
    mean_acc_scores = np.mean(mean_acc_scores)
    iu_scores = np.mean(iu_scores)
    freq_scores = np.mean(freq_scores)
    delayPrint("Checking {} file...".format(f), PRINT_SECONDS)
    if(isfile(f)):
        file = open(f, "a")
        delayPrint("Writing mean results file...", PRINT_SECONDS)
        file.write("Mean Results:\n")
        file.write("\t\tLoss: {}\n".format(loss))
        file.write("\t\tOverall accuracy (mean): {}\n".format(over_acc_scores))
        file.write("\t\tMean accuracy (mean): {}\n".format(mean_acc_scores))
        file.write("\t\tMean IU (mean): {}\n".format(iu_scores))
        file.write("\t\tFwavacc accuracy (mean): {}\n".format(freq_scores))
        delayPrint("Closing mean results file...", PRINT_SECONDS)
        file.close()
    else:
        delayPrint("{} log file does not exist!".format(f), PRINT_SECONDS)
        delayPrint("Creating {} file...".format(f), PRINT_SECONDS)
        delayPrint("Writing mean results file...", PRINT_SECONDS)
        file = open(f, "a+")
        file.write("Mean Results:\n")
        file.write("\t\tLoss: {}\n".format(loss))
        file.write("\t\tOverall accuracy (mean): {}\n".format(over_acc_scores))
        file.write("\t\tMean accuracy (mean): {}\n".format(mean_acc_scores))
        file.write("\t\tMean IU (mean): {}\n".format(iu_scores))
        file.write("\t\tFwavacc accuracy (mean): {}\n".format(freq_scores))
        delayPrint("Closing mean results file...", PRINT_SECONDS)
        file.close()
    return {'loss_a' : loss, 'over_acc_m' : over_acc_scores, 'mean_acc_m' : mean_acc_scores, 'iu_m' : iu_scores, 'freq_m' : freq_scores}

#### loop

In [67]:
def loop(paintings_path, labels_path, paintings, current_painting):
    n = input("Enter number of images to segment: ")
    index = paintings.index(current_painting.split(JPG_FILETYPE)[0])
    end = len(paintings) - 1
    last = index+int(n)
    if(index != 0):
        if(index == end):
            last = end
        else:
            index += 1
            last += 1
    delayPrint("Choose one of the following:\n", PRINT_SECONDS)
    delayPrint("[1] Segmentation\t\t[2] Validation", PRINT_SECONDS)
    option = int(input("Your answer [1/2]: "))
    while option != 1 and option != 2:
        option = int(input("Your answer [1/2]: "))
    for x in range(index, last):
        current_painting_path = paintings_path + "/" + paintings[x] + JPG_FILETYPE
        current_label_path = labels_path + "/" + paintings[x] + PNG_FILETYPE
        delayPrint(current_painting_path, PRINT_SECONDS)
        createCurrentLog(paintings[x], OUTPUT_FOLDER) if option is 1 else createCurrentLog(paintings[x], OUTPUT_FOLDER_VALIDATION)
        start_time = datetime.datetime.now()
        delayPrint("---------- Start Time - {:s} ----------".format(str(start_time)), PRINT_SECONDS)
        delayPrint("Index of painting: {}".format(x), PRINT_SECONDS)
        delayPrint("Last expected painting index: {}".format(last - 1), PRINT_SECONDS)
        delayPrint("", REVIEW_SECONDS)
        try:
            if option == 1:
                segmentation(current_painting_path, paintings[x])
            else:
                results = validation(current_painting_path, current_label_path, paintings[x])
                validationResultsLog(current_painting_path, results)
        except:
            error = sys.exc_info()[0]
            print("Error {} found! Writing error log.".format(error))
            writeErrorFile(current_painting_path, error)
        end_time = datetime.datetime.now()
        delayPrint("---------- End Time - {:s} ----------".format(str(end_time)), PRINT_SECONDS)
        elapsed_time = end_time - start_time
        delayPrint("Elapsed time is: {:s}".format(str(elapsed_time)), REVIEW_SECONDS)
        delayPrint("Index of painting: {}".format(x), PRINT_SECONDS)
        delayPrint("Last expected painting index: {}".format(last - 1), PRINT_SECONDS)
        delayPrint("", REVIEW_SECONDS)
        if int(n) > 1 and x != last - 1: # rest (60 seconds) if images are greater to 1 and no rest if last image
            time.sleep(REST_SECONDS)
        # if x == last - 1:
        writeResume(current_painting_path)
    if option is 2:
        mean_scores = computeMeanScore()
        delayPrint("Loss: {}".format(mean_scores["loss_a"]), PRINT_SECONDS)
        delayPrint("Overall Accuracy (mean): {}".format(mean_scores["over_acc_m"]), PRINT_SECONDS)
        delayPrint("Mean Accuracy (mean): {}".format(mean_scores["mean_acc_m"]), PRINT_SECONDS)
        delayPrint("Mean IU (mean): {}".format(mean_scores["iu_m"]), PRINT_SECONDS)
        delayPrint("Fwavacc Accuracy (mean): {}".format(mean_scores["freq_m"]), PRINT_SECONDS)
    delayPrint(endSession(), PRINT_SECONDS)

#### segmentation

In [68]:
def segmentation(path, current_painting):
    # the demo image is "2007_000129" from PASCAL VOC

    # load image, switch to BGR, subtract mean, and make dims C x H x W for Caffe
    # im = Image.open('demo/image.jpg')
    # path = "demo/Trials/twice.jpg"
    im = Image.open(path)
    if checkImageSize1000000(im):
        writeErrorFile(path, ERROR_ABOVE+dm)
    else:
        # reshape input layer from dimensions of image H x W
        reshapeInputLayer(im, 7)
        delayPrint("Starting to segment the image -- {} -- in {} seconds".format(current_painting, REVIEW_SECONDS), PRINT_SECONDS)
        # delay for 5 seconds for reviewing of image name
        time.sleep(REVIEW_SECONDS)
        in_ = np.array(im, dtype=np.float32)
        # print(in_)
        in_ = in_[:,:,::-1]
        # time.sleep(120)
        in_ -= np.array((104.00698793,116.66876762,122.67891434))
        in_ = in_.transpose((2,0,1))

        # Own code:
        # Set mode to CPU since GPU can't handle much memory
        caffe.set_mode_cpu()
        # load net
        net = caffe.Net('voc-fcn8s/test.prototxt', 'voc-fcn8s/fcn8s-heavy-pascal.caffemodel', caffe.TEST)
        # shape for input (data blob is N x C x H x W), set data
        net.blobs['data'].reshape(1, *in_.shape)
        net.blobs['data'].data[...] = in_

        # print("Stopping...")
        # return
        

        # run net and take argmax for prediction
        net.forward()
        out = net.blobs['score'].data[0].argmax(axis=0)

        # visualize segmentation in PASCAL VOC colors
        voc_palette = vis.make_palette(21)
        out_im = Image.fromarray(vis.color_seg(out, voc_palette))
        # image_pixels = ""
        # ycount_gt = 0
        # ycount_out = 0
        # for y in gt_:
        #     for x in y:
        #         # image_pixels += str(x)
        #         if x == 4:
        #             ycount_gt += 1
        #     # image_pixels += "\n"
        # for y in out_:
        #     for x in y:
        #         # image_pixels += str(x)
        #         if x == 4:
        #             ycount_out += 1
            # image_pixels += "\n"
        # print("Accuracy of 4: {:.5f}".format(ycount_out/ycount_gt))
        # print(ycount)
        # # print(image_pixels)
        # with open("test.txt", "a+") as file:
        #     file.writelines(image_pixels)
        # out_im.save('demo/output.png')
        out_im.save('demo/%s/output_%s.png'%(OUTPUT_FOLDER, current_painting.split(JPG_FILETYPE)[0]))
        logfile = "demo/"+OUTPUT_FOLDER+"/"+current_painting+".log"
        masked_im = Image.fromarray(vis.vis_seg(im, out, voc_palette, 0.5, logfile))

        # print extracted colors of original image
        vis.extractColors(path, logfile)

        # masked_im.save('demo/visualization.jpg')
        masked_im.save('demo/%s/output_%s.jpg'%(OUTPUT_FOLDER, current_painting))

#### valdation

In [69]:
def validation(painting_path, label_path, current_painting):
    im = Image.open(painting_path)
    gt = Image.open(label_path)
    gt = gt.resize(gt.size, Image.ANTIALIAS)
    if checkImageSize1000000(im):
        writeErrorFile(painting_path, ERROR_ABOVE+dm)
    else:
        # reshape input layer from dimensions of image H x W
        reshapeInputLayer(im, 7)
        reshapeLabelLayer(gt, 15)
        delayPrint("Starting to validate the image -- {} -- in {} seconds".format(current_painting, REVIEW_SECONDS), PRINT_SECONDS)
        # delay for 5 seconds for reviewing of image name
        time.sleep(REVIEW_SECONDS)
        in_ = np.array(im, dtype=np.float32)

        # try ground truth
        gt_ = np.array(gt, dtype=np.float32)
        # gt_ = gt_[np.newaxis, ...]
        # print(in_)
        in_ = in_[:,:,::-1]
        # print(in_)
        # print(gt_)
        # gt_ = gt_[:,:,::-1]
        # time.sleep(120)
        in_ -= np.array((104.00698793,116.66876762,122.67891434))
        in_ = in_.transpose((2,0,1))

        # gt_ -= np.array((104.00698793,116.66876762,122.67891434))
        # gt_ = gt_.transpose((2,0,1))

        # Own code:
        # Set mode to CPU since GPU can't handle much memory
        caffe.set_mode_cpu()
        # load net
        net = caffe.Net('voc-fcn8s/test.prototxt', 'voc-fcn8s/fcn8s-heavy-pascal.caffemodel', caffe.TEST)
        # shape for input (data blob is N x C x H x W), set data
        # net.blobs['label'].data = gt_ 
        
        # print(net.blobs)
        
        net.blobs['data'].reshape(1, *in_.shape)
        net.blobs['data'].data[...] = in_
        # net.blobs['label'].reshape(1, *gt_.shape)
        net.blobs['label'].data[...] = gt_

        # print("Stopping...")
        # return
        

        # run net and take argmax for prediction
        net.forward()
        
        out = net.blobs['score'].data[0].argmax(axis=0)
        voc_palette = vis.make_palette(21)
        out_im = Image.fromarray(vis.color_seg(out, voc_palette))
        out_im.save('demo/%s/output_%s.png'%(OUTPUT_FOLDER_VALIDATION, current_painting.split(JPG_FILETYPE)[0]))
        logfile = "demo/"+OUTPUT_FOLDER_VALIDATION+"/"+current_painting+".log"
        masked_im = Image.fromarray(vis.vis_seg(im, out, voc_palette, 0.5, logfile))

        # print extracted colors of original image
        vis.extractColors(painting_path, logfile)

        # masked_im.save('demo/visualization.jpg')
        masked_im.save('demo/%s/output_%s.jpg'%(OUTPUT_FOLDER_VALIDATION, current_painting))
        # out_ = np.array(out, dtype=np.uint8)
        # delayPrint("Ground truth image array: {}".format(gt_.flatten().shape))
        # print("Segmentation result image array: {}".format(out_.flatten().shape))
        
        # check accuracy
        # val = np.loadtxt('demo/valid.txt', dtype=str)
        val = np.array([1], dtype=np.uint8)

        # reshaping blobs to 1-dimension
        # net.blobs['data'].reshape(1,)
        # net.blobs['label'].reshape(1,)
        return score.do_seg_tests(net, 1, False, val, layer='score', gt='label')

### Main process here:

In [72]:
# main process
delimiter = "/"
fp_resume = readResume()

# current painting
current_painting = fp_resume.split("/")
current_painting = current_painting[len(current_painting) - 1].rstrip() # rstrip() for removing white spaces and new line instances

# paintings_path
paintings_path = fp_resume.split("/")
paintings_path.remove(paintings_path[len(paintings_path) - 1])
paintings_path = delimiter.join(paintings_path)

# labels_path
labels_path = fp_resume.split("/")
labels_path.remove(labels_path[len(labels_path) - 1])
labels_path.remove(labels_path[len(labels_path) - 1])
labels_path.append("SegmentationClassPNG")
labels_path = delimiter.join(labels_path)

paintings = getPaintings(paintings_path)
# print(paintings)
loop(paintings_path, labels_path, paintings, current_painting)

Checking demo/resume.txt file...
Reading file...
Last segmented image: demo/test/presentation_voc/JPEGImages/albert-bierstadt_bison-with-coyotes-in-the-background.jpg
Closing file...
Enter number of images to segment: 1
Choose one of the following:

[1] Segmentation		[2] Validation
Your answer [1/2]: 2
demo/test/presentation_voc/JPEGImages/albert-bierstadt_bison-with-coyotes-in-the-background.jpg
Resuming demo/output_validate/albert-bierstadt_bison-with-coyotes-in-the-background.log file
---------- Start Time - 2018-08-31 14:12:52.220302 ----------
Index of painting: 0
Last expected painting index: 0

Checking if image {width, height} has 1,000,000 and above pixels...
Images (H x W) has dimensions 352 x 502
Image {height, width} has below 1000 pixels - (352 x 502)
Checking voc-fcn8s/test.prototxt file...
Reading file...
Reshaping input layer...
    shape { dim: 1 dim: 3 dim: 352 dim: 502 }

Writing file...
Closing file...
Checking voc-fcn8s/test.prototxt file...
Reading file...
Reshapi

/home/butchersix/fcn.berkeleyvision.org/score.py:115: RuntimeWarning: invalid value encountered in true_divide
  mean_acc = np.diag(hist) / hist.sum(1)


>>>2018-08-31 14:13:36.507173 Iteration: 1 Mean Accuracy: 0.49793025852876227
>>>2018-08-31 14:13:36.724853 Iteration: 1 Mean IU: 0.19220665570077705


/home/butchersix/fcn.berkeleyvision.org/score.py:119: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))


>>>2018-08-31 14:13:36.926132 Iteration: 1 Fwavacc: 0.9226096098565758
Checking demo/results.log file...
Writing results file...
Closing results file...
---------- End Time - 2018-08-31 14:13:37.875380 ----------
Elapsed time is: 0:00:45.655078
Index of painting: 0
Last expected painting index: 0

Checking demo/resume.txt file...
Saving last segmented image path...
Writing file...
Closing file...
Checking demo/results.log file...
Writing mean results file...
Closing mean results file...
Loss: 53080.94921875
Overall Accuracy (mean): 0.956044398945646
Mean Accuracy (mean): 0.49793025852876227
Mean IU (mean): 0.19220665570077705
Fwavacc Accuracy (mean): 0.9226096098565758

-------------------- END SESSION - 2018-08-31 14:13 -------------------------

